

<p style="font-size:300%; background-color:pink; color:blue; text-align:center;line-height : 80px; margin : 0; padding : 0;">
Entrega 3 - Proyecto FInal </p>
<p style="font-size:240%; background-color:pink; color:red; text-align:center;line-height : 60px; margin : 0; padding : 0;">
    
Técnicas de Inteligencia Artificial - Visión de Máquina </p1>

<p style="font-size:200%; text-align:center; line-height : 40px;  margin-top : 0; margin-bottom : 0; "> 
  <br>Est. Brian Camilo Valencia Peña (bcvalenciap@unal.edu.co)
  <br>Est. Esteban Ladino Fajardo (eladinof@unal.edu.co)
</p>

<p style="font-size:180%; text-align:center; line-height : 30px;  margin-top : 0; margin-bottom : 0; "> <br><br>INGENIERÍA MECATRÓNICA Y ELECTRÓNICA</p>
<p style="font-size:180%; text-align:center; line-height : 30px; margin-top : 0; "> Facultad de Ingeniería</p>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> Universidad Nacional de Colombia Sede Bogotá</p>
<br>
<img src="../figs/Escudo_UN.png",width=100,height=100>

<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> <br>28 de Junio de 2022</p>

In [5]:
# Librerias
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy import stats as st
from scipy.cluster import vq
from matplotlib.colors import ListedColormap
import time as t
print ("OpenCV Version : %s " % cv2.__version__)
from scipy import signal
from scipy import misc
from sklearn import neighbors, datasets
%matplotlib inline


## Aplicación de algoritmos de calsificación supervisado
Detección de Cacao enfermo o sano, muestras de color rojo y amarillo.


In [6]:
# importación de características y datos de procesamiento
Xpoblacional=np.load('../data/caracteristicasNormalizadasPCA.npy')
imagenesProcesadas=np.load('../data/listaImagenesProcesadas.npy')
nombreCaracteristicas=np.load('../data/nombreCaracteristicasNormalizadasPCA.npy')

#print(f'Imágenes procesadas:\n {imagenesProcesadas}')
#print(f'Nombre de características finales\n{nombreCaracteristicas}')
#print(f'caracteristicasNormalizadasPCA\n{Xpoblacional}')

Imágenes procesadas:
 ['monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito'
 'fito' 'fito' 'fito' 'fito' 'saludable' 'saludable' 'saludable'
 'saludable' 'saludable' 'saludable' 'saludable' 'saludable' 'saludable'
 'saludable']
Nombre de características finales
['Moda R' 'Moda G' 'Moda B' 'Moda Gris' 'Media R' 'Media G' 'Media B'
 'Media Gris' 'mínimo R' 'mínimo G' 'mínimo B' 'mínimo Gris' 'máximo R'
 'máximo G' 'máximo B' 'máximo Gris' 'Momento HU 1' 'Momento HU 2'
 'Momento HU 3' 'Momento HU 4' 'Momento HU 5' 'Momento HU 6'
 'Momento HU 7' 'Redondez']
caracteristicasNormalizadasPCA
[[5.21568627e-01 4.19607843e-01 4.62745098e-01 4.74509804e-01
  5.11328312e-01 3.96780049e-01 3.95690281e-01 4.29720066e-01
  3.92156863e-03 3.92156863e-03 3.92156863e-03 3.92156863e-03
  1.00000000e+00 9.96078431e-01 9.68627451e-01 9.92156863e-01
  3.87435724e-01 3.27410544e-01 1.47617426e-04 2.42949234e-04
  2.37957

En esta ocasión se va a clasificar si el cacao está o no enfermo. Si está enfermo la salida es 1 y si está sano la salida es 0.

### Entrenamiento con el perceptrón

Las siguientes líneas de código permiten encontrar el peso y bias.

Mediante un perceptrón se demoró 460 épocas en entrenarse. Lo importante fue que se logró tener un número de convergencia. Ahora, se pasa a analizar los resultados con las características de prueba.

In [7]:

#fig = plt.figure(dpi=200,figsize=(8,2))
#ax = fig.add_subplot(1,1,1)
#table_data=[
#    ['', 'Enfermo','Sano'],
#    ['Enfermo', VA,FN],
#    ['Sano', FA,VR]]
#table = ax.table(cellText=table_data, loc='center')
#table.set_fontsize(12)
#ax.axis('off')
#ax.set_title('Matriz de confusión')
#plt.show()

#### Valores estadísticos

A continuación se presentan los valores estadísticos

La exactitud fue cercana al 80% y en general es un buen resultado para obtenerse con un perceptrón.

### Entrenamiento con adaline



Los resultados fueron los mismos que los obtenidos con el perceptrón pero con mayores cálculos.